In [1]:
import sys
sys.path
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import bs4
import time
import os
import re
import csv
DEBUG = 1
from datetime import datetime, timedelta
import dateparser
import calendar

In [2]:
def convertnumber(number):
    if 'k' in number:
        n=float(number.replace('k', '').replace(' ', ''))*1000
    elif 'm' in number:
        n=float(number.replace('m', '').replace(' ', ''))*1000000
    else:
        n=number
    return int(n)  

In [3]:
def scrolldown(browser):        
    src_updated = browser.page_source
    src = ""
    while  src != src_updated:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        src = src_updated
        src_updated = browser.page_source  

In [4]:
def connectchrome():
    # instantiate a chrome options object so you can set the size and headless preference
    options = Options()
    options.add_argument('--headless')
    options.add_argument('log-level=3')
    options.add_argument("--incognito")
    driver = webdriver.Chrome(options=options)
    driver.execute_script('window.scrollTo(0, 0);')
    # download the chrome driver from https://sites.google.com/a/chromium.org/chromedriver/downloads
    # and put it in the current directory
    chromedriver = "/home/youcef/Documents/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver     
    browser = webdriver.Chrome(options=options) 
    browser.get('https://www.quora.com/')
    time.sleep(2)
    return browser

In [5]:
def getTopic():
    file_question_topics = open((os.path.join('/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping'+'/topic.txt')), mode='r', encoding='utf-8')
    lines=file_question_topics.readlines()
    line=lines[0]
    topic=line.strip()
    return str(topic.strip('\n'))
    

In [6]:
def crawlURL():
    # read topics form a file
    print(sys.path[0])
    file_question_topics = open((os.path.join('/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping'+'/topic.txt')), mode='r', encoding='utf-8')
    cc=0
    driver=connectchrome()
    lines=file_question_topics.readlines()
    parsing_loop=True
    print(len(lines))
    while parsing_loop:
        line=lines[cc]
        cc+=1
        topic=line.strip()
        if (topic.startswith("#")):
            continue

       # give the url to scrape

        try:
            print('###################')
            print('starting new topic: ' + str(topic))
            url = "https://www.quora.com/topic/" + topic.strip('\n') + "/all_questions"
            print(str(url))
            driver.get(url)
        except Exception as e0:
            print('exception e0')
            print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e0).__name__, e0)
            continue     


        # define pause time for browser
        SCROLL_PAUSE_TIME = 2

        # get browser source
        html_source = driver.page_source
        question_count_soup = BeautifulSoup(html_source, 'html.parser')

        #  get total number of questions
        question_count_str = question_count_soup.find('a', attrs={'class': 'TopicQuestionsStatsRow'})

        if str(question_count_str)=='None':
            print('topic does not exist in Quora')
            continue

        question_count = question_count_str.contents[0].text 
        if 'k' in str(question_count):
            question_count = str(question_count).replace('k','')
            question_count = int(float(question_count)*1000)
            #print(type(question_count))
        print('number of questions for this topic : '+ str(question_count))

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        question_set = set()
        # infinite while loop, break it when you reach the end of the page or not able to scroll further.
        if int(question_count)>10:
            start_time_sd = time.time()
            max_time=  int(question_count)*0.25
            if int(question_count)> 8000:
                max_time=1800
            while True:
                i = 0
                # try to scroll 5 times in case of slow connection
                while True:
                    # Scroll down to one page length
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    # Wait to load page
                    time.sleep(SCROLL_PAUSE_TIME)

                    # get page height in pixels
                    new_height = driver.execute_script("return document.body.scrollHeight")

                    # break this loop when you are able to scroll futher
                    if new_height != last_height:
                        break
                    if i==1:
                        SCROLL_PAUSE_TIME = 4   
                    if i==2:
                        if int(question_count)>500 and int(question_count)<2000:
                            SCROLL_PAUSE_TIME = 7
                        elif int(question_count)>2000:
                            SCROLL_PAUSE_TIME = 15    
                        elif int(question_count)<500:
                            break
                    if i==3:
                        break
                    i += 1
            SCROLL_PAUSE_TIME = 2    
            # not able to scroll further, break the infinite loop
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            total_time=time.time() - start_time_sd
            if total_time>max_time:
                print('max time exceeded')
                break



        print('getting ressources ...')
        # get html page source
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        # question_link is the class for questions
        question_link = soup.find_all('a', attrs={'class': 'question_link'}, href=True)

        # add questions to a set for uniqueness
        for ques in question_link:
            question_set.add(ques)


        # write content of set to a file called question_urls.txt
        questions_directory = 'topics/'
        os.makedirs('topics/', exist_ok=True)
        file_name = questions_directory + '/' + topic.strip('\n') + '_question_urls.txt'
        file_question_urls = open(file_name, mode='w', encoding='utf-8')
        writer = csv.writer(file_question_urls)
        for ques in question_set:
            link_url = "http://www.quora.com" + ques.attrs['href']
            #print(link_url)
            writer.writerows([[link_url]])

        #sleeping each 300 requests
        if cc % 300== 0:
            print('quitting chrome')
            driver.quit()
            time.sleep(6)
            driver=connectchrome()
        #loop_limit can be adjusted based on the number of requests we want to send 
        loop_limit=2
        if cc>loop_limit:
            parsing_loop=False

    # finish time
    end_time = datetime.datetime.now()
    print(end_time-start_time)

In [7]:
def extractQuestionTitle(driver):
    try:
        question_title = driver.find_element_by_xpath('.//span[contains(@class, "ui_qtext_rendered_qtext")]').text
    except Exception as e:
        question_title = 'error'
    return question_title

In [8]:
def extractAnswerCount(driver):
    try:
        answer_count = driver.find_element_by_xpath("//div[@class = 'answer_count']")
        text_count = answer_count.text.split()
        count = int((text_count[0].split('+'))[0])
    except:
        count = "error"
    return count

In [9]:
def extractTopics(driver):
    topic_list = []
    
    try:
        topic_area = driver.find_element_by_xpath(".//div[contains(@class,'QuestionTopicHorizontalList')]")
        
    except:
        return topic_list
    
    try:
        topics = topic_area.find_elements_by_xpath(".//span[@class = 'TopicName TopicNameSpan']")
        for topic in topics:
            topic_list.append(topic.text)
    except:
        topic_list = ['error']
           
    return topic_list

In [10]:
def scrollDownPage(driver, speed):
    current_scroll_position, new_height= 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [1]:
def expandAllAnswers(driver):
    hidden_list = driver.find_elements_by_xpath(".//div[@class='ui_qtext_truncated ui_qtext_truncated_regular']/div[@class='ui_qtext_truncated_text']")
    #print('Number of hidden answer:',len(hidden_list))
    for hidden in hidden_list:
        hidden = hidden.find_element_by_xpath(".//a[@class='ui_qtext_more_link']")
        webdriver.ActionChains(driver).move_to_element(hidden).click(hidden).perform()

In [ ]:
def extractRespondContent(driver):
    try:
        respond_content = driver.find_element_by_xpath(".//div[@class = 'q-text']").text
    except Exception as e:
        repond_content = 'error'
        
    return respond_content

In [12]:
def extractRespond(driver):
    check_ans_count = 0
    answers_list = {}
    name_list = []
    upvote_list = []
    ans_list = []
    date_list = []
    view_list = []
    error_list = []
    question_title = ''
    check_ans_count = 0
    try: 
        respond_list = driver.find_elements_by_xpath(".//div[contains(@class, 'AnswerBase') and contains(@class, 'AnswerBase') and contains(@class, 'Toggle') and contains(@class, 'Answer')]")
        for respond in respond_list:
            # Name
            ans = {}
            try:
                user_name = respond.find_element_by_xpath('.//a[@class = "user"]')
                name_list.append(user_name.text)
                ans['user_name'] = user_name.text
            except Exception:
                user_name = respond.find_element_by_xpath('.//span[contains(@class, "anon_user")]')
                name_list.append(user_name.text)
                ans['user_name'] = user_name.text
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                name_list.append('')
                ans['user_name'] = 'error'
                print(error_list[-1])

            # Date
            try:
                date = respond.find_element_by_xpath(".//a[@class = 'answer_permalink']")
                date_split = date.text.split('Answered ')
                question_date = date_split[1]
                date_list.append(question_date)
                ans['date'] = question_date
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                date_list.append('')
                ans['date'] = 'error'
                print(error_list[-1])

            # Features that show after expand
            try:
                expanded_answer = respond.find_element_by_xpath(".//div[contains(@class,'ExpandedAnswer')]")
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                print(error_list[-1])
                continue

            # Answer
            try:
                answer = expanded_answer.find_element_by_xpath(".//div[@class='ui_qtext_expanded']/span[@class='ui_qtext_rendered_qtext']")
                ans_list.append(answer.text)
                ans['answer'] = answer.text
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                ans_list.append('')
                ans['answer'] = 'error'
                print(error_list[-1])

            # Answer upvote
            try:
                upvote = expanded_answer.find_element_by_xpath(".//a[@class='VoterListModalLink']")
                res = [int(i) for i in upvote.text.split() if i.isdigit()]
                upvote_list.append(res[0])
                ans['upvote'] = res[0]
            except Exception:
                upvote_list.append(0)
                ans['upvote'] = 0
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                upvote_list.append(0)
                ans['upvote'] = 'error'
                print(error_list[-1])

            # Answer view
            try:
                answer_footer = expanded_answer.find_element_by_xpath(".//div[contains(@class, 'AnswerFooter')]")
                view_text = (answer_footer.text.split())[0]
                if 'k' in view_text:
                    view = float((view_text.split('k'))[0]) * 1000
                elif 'm' in view_text:
                    view = float((view_text.split('m'))[0]) * 1000000
                else:
                    view = int(view_text)
                view_list.append(view)
                ans['view'] = view
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                view_list.append(0)
                ans['view'] = 'error'
                print(error_list[-1])

            print()    
            check_ans_count += 1
            print(type(ans))
            if check_ans_count > len(answers_list):
                answers_list[str(len(answers_list))] = ans
        print("Respond collected:", check_ans_count)

        return answers_list
    
    except Exception as e:
        error_list.append([e, question_title, check_ans_count])
        print(error_list[-1])
        return check_ans_count, name_list, upvote_list, ans_list, date_list, view_list, error_list
    
    

In [13]:
def scrollDownPageFaster(driver):
    current_scroll_position, new_height= 0, 1
    speed = 5
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [14]:
def getQuestionAddedBy(driver):
    try:
        last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]
        user_name = last_item.find_element_by_xpath(".//span[@class = 'anon_user']").text
    except:
        try:
            last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]
            user_name = last_item.find_element_by_xpath(".//a[@class = 'user']").text
        except:
            return
    return user_name

In [15]:
def extractCreateTime(driver):
    try:
        last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]
        question_data = last_item.find_element_by_xpath(".//p[@class = 'log_action_bar']")
        question_data_split = question_data.text.split('· ')
        question_date = question_data_split[1]
    except:
        question_date = ''
    return question_date

In [ ]:
def extractDate(driver):
    try:
        log = driver.find_element_by_xpath(".//p[@class = 'log_action_bar']")
        date = log.text.split('· ')[1]
    except:
        date = 'error'
    
    return date

In [ ]:
def extractAuthor(driver):
    block_header = driver.find_element_by_xpath (".//div[@class = 'feed_item_activity']")
    try:
        user_name = block_header.find_element_by_xpath(".//span[@class = 'anon_user']").text
    except Exception:
        user_name = block_header.find_element_by_xpath(".//a[@class = 'user']").text
    except Exception as e:
        user_name = 'error'
    
    return user_name

In [ ]:
def log_stat_extraction(driver, driver2):
    # return element
    respond_author_list = []
    respond_content_list = []
    respond_upvote_list = []
    respond_view_list = []
    respond_date_list = []
    respond_count = 0
    question_auther = ''
    question_create_time = ''
    
    try:
        block_list = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")
    except:
        respond_author_list =  ['error']
        respond_content_list = ['error']
        respond_upvote_list = ['error']
        respond_view_list = ['error']
        respond_date_list = ['error']
        respond_count = 0
        question_auther = 'error'
        question_create_time = 'error'
        
        return respond_author_list, respond_content_list, respond_upvote_list, respond_view_list, respond_date_list, respond_count, question_auther, question_create_time
        
    for block in block_list:
        block_error = []
        
        block_header = block.find_element_by_xpath (".//div[@class = 'feed_item_activity']").text
        
        try:
            block_revision = block.find_element_by_xpath (".//div[@class = 'revision']").text
        except:
            continue
            
        if 'answer has since been deleted' in block_revision:
            continue
        elif 'deleted' in block_header:
            continue
        
        if 'Answer' in block_header:
            # click the answer link
            try:
                url2 = block.find_element_by_xpath (".//div[@class = 'feed_item_activity']/a").get_attribute('href')
                driver2.get(str(url2))
                respond_count += 1
            except Exception as e:
                continue
                
            # extract author name(log page and driver 1)
            respond_author = extractAuthor(block)
            respond_author_list.append(respond_author)
            
            # extract content
            respond_content = extractRespondContent(driver2)
            respond_content_list.append(respond_content)
                
            # extract views and upvote
            respond_view, respong_upvote = extractRespondViewAndUpvote(driver2)
            respond_view_list.append(respond_view)
            respond_upvote_list.append(respong_upvote)
                
            # extract date(log page and driver 1)
            respond_date = extractDate(block)
            respond_date_list.append(respond_date)
            
        elif 'Question added by' in block_header:
            # extract author name
            question_author = extractAuthor(block)
            
            # extract date
            question_create_time = extractDate(block)

    return respond_author_list, respond_content_list, respond_upvote_list, respond_view_list, respond_date_list, respond_count, question_author, question_create_time



In [ ]:
def extractNoOfFollowerAndView(driver):
    try:
        stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
        no_of_follower = int((stats[0].text.split(' '))[0])
        no_of_view = (stats[1].text.split(' '))[0]
    except:
        no_of_follower = 0
        no_of_view = 0
        
    return no_of_follower, no_of_view

In [16]:
def extractNoOfFollower(driver):
    try:
        stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
        no_of_follower = int((stats[0].text.split(' '))[0])
    except:
        no_of_follower = 'error'
    return no_of_follower

In [17]:
def extractNoOfView(driver):
    try:
        stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
        no_of_view = (stats[1].text.split(' '))[0]
    except:
        no_of_view = 'error'
    return no_of_view

In [ ]:
def extractRespondViewAndUpvote(driver):
    try:
        respond_stat = driver.find_element_by_xpath(".//div[@class = 'q-text qu-mt--small qu-color--gray_light qu-fontSize--small qu-passColorToLinks']").text

        view_text = respond_stat.split(' view')[0]
        if 'k' in view_text:
            respond_view = int(float((view_text.split('k'))[0]) * 1000)
        elif 'm' in view_text:
            respond_view = int(float((view_text.split('m'))[0]) * 1000000)
        else:
            respond_view = int(view_text)

        if 'Upvoter' in respond_stat:
            upvote_text = (respond_stat.split('View ')[1]).split(' ')[0]
            if 'k' in upvote_text:
                respond_upvote = int(float((upvote_text.split('k'))[0]) * 1000)
            elif 'm' in upvote_text:
                respond_upvote = int(float((upvote_text.split('m'))[0]) * 1000000)
            else:
                respond_upvote = int(upvote_text)
        else:
            respond_upvote = 0

    except:
        respond_view = 0
        respond_upvote = 0
        
    return respond_view, respond_upvote

In [18]:
def extractQuestion(driver, url):
    # Extract Question
    data = {}
    scrollDownPage(driver)  
    expandAllAnswers(driver)
    data['question_title'] = extractQuestionTitle(driver)
    data['answer_count'] = extractAnswerCount(driver)
    data['topic_list'] = extractTopics(driver)
    data['respond'] = extractRespond(driver)
    driver.get(url+'/log')
    scrollDownPageFaster(driver)
    data['question_author'] = getQuestionAddedBy(driver)
    data['create_time'] = extractCreateTime(driver)
    data['no_of_follower'] = extractNoOfFollower(driver)
    data['no_of_view'] = extractNoOfView(driver)
    return data

In [19]:
def crawlQuestionData(file):
    start_time = time.time()
    #if (DEBUG): #print ("In crawlQuestionData...")
    
    # Open question url file
    file_question_urls = open(file, mode = 'r')
    topic = str(getTopic())
    topic = topic + "_answers.txt"

    #file_users = open(os.path.join(sys.path[0]+"users.txt", mode = 'a'))
    current_lines = file_question_urls.readlines()
    i=-1
    browser= connectchrome()
    k=0
    loop=True
    #for current_line in current_lines:
    dictdata = {}
    while  loop :
        try:
            current_line= current_lines[k]
        except:
            break
        print('processing question number  : '+ str(k))
        k+=1
        if '/unanswered/' in str(current_line):
            print('answer is unanswered')
            continue     
        print ("*************************************************************************")
        if (DEBUG): print(current_line)
        question_id = current_line
        browser.get(current_line)
        data = extractQuestion(browser, current_line)
        print(type(data))
        print(data)
        dictdata[str(len(dictdata))] = data
        print(type(dictdata))
        
        j = json.dumps(dictdata)
        # we save all retrieved answers into answers.txt file
        with open(topic, 'w') as file_answers:
            file_answers.write(j)
            file_answers.close()
    
        print ("*************************************************************************")
        # In our usecase 74256 is the loop limit value
        if k>=1134 :
           loop= False
    print("--- %s seconds ---" % (time.time() - start_time))        
    browser.quit()

In [20]:
def main():
    print(os.getcwd())
    #crawlURL()
    # We merged all Questions urls crawled by 1-Questions_URLs_crawler.py into one file (quora_urls.txt)
    crawlQuestionData((os.path.join(os.getcwd()+"/topics/" + getTopic() + "_question_urls.txt")))
    return 0

if __name__ == "__main__": main()    

/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping
processing question number  : 0
answer is unanswered
processing question number  : 1
*************************************************************************
http://www.quora.com/How-are-you-maintaining-your-relationships-during-the-lockdown-period-of-21-days-during-this-Pandemic-2020

Respond collected: 0
<class 'dict'>
{'question_title': 'How are you maintaining your relationships during the lockdown period of 21 days during this Pandemic 2020?', 'answer_count': 'error', 'create_time': 'Apr 9, 2020', 'question_author': 'Shubham Das', 'no_of_follower': 1, 'no_of_view': '183', 'respond': {}, 'topic_list': ['COVID-19 Quarantines & Restrictions', 'Pandemics', '2020 (year)', 'Relationships', 'Infectious Diseases', 'COVID-19 (2019-2020)', 'Dating and Relationships', 'Interpersonal Interaction']}
<class 'dict'>
*************************************************************************
processing quest


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'question_title': 'How has coronavirus affected your long distance relationship?', 'answer_count': 1, 'create_time': 'Apr 1, 2020', 'question_author': 'Stewart Bruinsma', 'no_of_follower': 4, 'no_of_view': '690', 'respond': {'0': {'user_name': 'Janie Carr', 'answer': 'The pandemic has affected us in multiple ways.\nMy partner came to visit me in the UK from Florida in March. We were in London when the USA announced they were implementing a travel ban on the UK. He had to reschedule his flight to much earlier than planned. I didn’t even get to show him Big Ben.\nHowever, we were both asked to self-isolate after he’d returned home. This meant we spent a lot of time together online where he would usually be working 40 hours a week. While I would rather have had the time with him in person, it has been nice to be able to speak to him for longer in the daytime for a while.\nMy fear now is when I will see him again. Will I be able to trave

<class 'dict'>
{'question_title': 'Why is it more common for most polyamory relationships to be MFF rather than MFM?', 'answer_count': 4, 'create_time': 'Mar 31, 2020', 'question_author': 'Catherine I', 'no_of_follower': 1, 'no_of_view': '7,349', 'respond': {}, 'topic_list': []}
<class 'dict'>
*************************************************************************
processing question number  : 16
*************************************************************************
http://www.quora.com/Do-most-people-go-through-multiple-relationships-before-marriage-or-finding-the-one


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'question_title': 'Do most people go through multiple relationships before marriage or finding "the one"?', 'answer_count': 1, 'create_time': 'Apr 7, 2020', 'question_author': 'Thao Dao', 'no_of_follower': 2, 'no_of_view': '1,860', 'respond': {'0': {'user_name': 'Frank Clay', 'answer': "Some people by choice, some have no control over what happens.\nEach relation


<class 'dict'>

<class 'dict'>
Respond collected: 2
<class 'dict'>
{'question_title': 'Is it self-deception to be in a relationship when one believes that none of it last?', 'answer_count': 2, 'create_time': 'Apr 14, 2020', 'question_author': 'Racoff Chat', 'no_of_follower': 1, 'no_of_view': '572', 'respond': {'1': {'user_name': 'Peter Ratcliffe', 'answer': "It’s not yourself that’s being deceived when you believe that none of it will last.\nYou are deceiving your lover who believes that all of it will last.\nYou owe them honesty.\nLove is a choice that must be made by two lovers, one can’t do it.\nThe belief that it will last a lifetime is part of all lifetime lover’s beliefs as they choose to stay with each other, one day at a time, for a lifetime.\nSee also: Peter Ratcliffe's answer to What are the first signs that make you understand that you are falling out of love with your partner?", 'upvote': 0, 'view': 38, 'date': 'Apr 16, 2020'}, '0': {'user_name': 'Myan Nguyen', 'answer': "


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'question_title': 'Do you wish you never found out your spouse or partner was cheating, or is it better to know?', 'answer_count': 1, 'create_time': 'Apr 6, 2020', 'question_author': 'Nicki Hansen', 'no_of_follower': 2, 'no_of_view': '828', 'respond': {'0': {'user_name': 'Jennifer Perez', 'answer': 'I’m glad I learned. It was exactly what we needed to fix our relationship because we were fighting and breaking up and fighting.\nI’m not saying I appreciated either my fiancé (at the time and now husband) or the whore that had been pursuing him doing that to me. But it wasn’t the deal breaker that I thought it would be. But I feel that’s because I learned he really didn’t give a damn about the whore and was simply using her in a car at lunch and being paid to get blowjobs, because she really was that desperate.\nShe was so Jerry Springer, I knew the moment I saw her she was no threat to our relationship. And when he confirmed what I alre

...

Respond collected: 0
<class 'dict'>
{'question_title': 'error', 'answer_count': 'error', 'create_time': '', 'question_author': None, 'no_of_follower': 'error', 'no_of_view': 'error', 'respond': {}, 'topic_list': []}
<class 'dict'>
*************************************************************************
processing question number  : 1111
*************************************************************************
http://www.quora.com/How-long-did-it-take-for-you-and-your-significant-other-to-know-you-wanted-to-be-with-each-other-for-life-How-long-did-you-take-before-you-both-admitted-it


<class 'dict'>

<class 'dict'>
Respond collected: 2
<class 'dict'>
{'question_title': 'How long did it take for you and your significant other to know you wanted to be with each other for life? How long did you take before you both admitted it?', 'answer_count': 2, 'create_time': 'Mar 31, 2020', 'question_author': 'Thao Dao', 'no_of_follower': 1, 'no_of_view': '1,053', 'respond': {'1': {'user_name': 'Car

<class 'dict'>
{'question_title': 'How was your first relationship like?', 'answer_count': 3, 'create_time': 'Apr 4, 2020', 'question_author': 'Khyati Sharma', 'no_of_follower': 1, 'no_of_view': '546', 'respond': {}, 'topic_list': ['Relationship Experiences', 'Relationships', 'Personal Experiences', 'International Relations', 'Love', 'Relationship Advice', 'Experiences in Life', 'Dating Advice', 'Dating and Relationships', 'Interpersonal Interaction']}
<class 'dict'>
*************************************************************************
processing question number  : 1114
*************************************************************************
http://www.quora.com/How-is-your-relationship-with-your-neighbor


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'question_title': 'How is your relationship with your neighbor?', 'answer_count': 1, 'create_time': 'Apr 16, 2020', 'question_author': 'Cindy Ha', 'no_of_follower': 2, 'no_of_view': '2,456', 'respond': {'0': {'user_name': 'Eli


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'question_title': 'Women always look for a better male partner than the current one. And if she is good looking, most likely she will be in multiple relationships. What is psychological explanation of this?', 'answer_count': 1, 'create_time': 'Mar 30, 2020', 'question_author': 'Nakul Pariker', 'no_of_follower': 0, 'no_of_view': '566', 'respond': {'0': {'user_name': 'Akash Panchal', 'answer': 'Does women always look a better male partner that the current one?? I’ve explained this in one of my answers. here is the link for your first doubt.\nhttps://www.quora.com/Do-all-gir...\nFirst, it is true that person always want better than now. A “nothing” man seeks for “something”, and the achievement of “something” makes him greedy for something more than current “something”. And this goes on.\nThis phenomena could not be recognized by most of the people, who cannot identify what and why they are doing, and they just flow.\nSecond, ‘good look

[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]

<class 'dict'>

<class 'dict'>

<class 'dict'>
Respond collected: 3
<class 'dict'>
{'question_title': 'Would you change or delete Facebook if your partner was insecure about it and needed a little piece of mind to stop turmoil in the relationship?', 'answer_count': 4, 'create_time': 'Apr 15, 2020', 'question_author': 'Jason Jones', 'no_of_follower': 2, 'no_of_view': '775', 'respond': {'2': {'user_name': 'Anne North', 'answer': 'N that is dictating to you if they are that jealous or needy or if you have done something for them to not trust you maybe it’s best to talk it over and see why it bothers them are you spending more time on fb than on them is there something you or them are hiding', 'upvote': 0, 'view': 14, 'date': 'Apr 15, 2020'}, '1': {'user_name': 'Kat Christopher'

Respond collected: 0
<class 'dict'>
{'question_title': 'Have you ever ghosted a date? Why did you do it?', 'answer_count': 'error', 'create_time': 'Apr 11, 2020', 'question_author': 'Daniele De Rossi', 'no_of_follower': 2, 'no_of_view': '830', 'respond': {}, 'topic_list': ['Dating Experiences', 'Ghosting', 'Dating Etiquette', 'Dating', 'Relationships', 'Personal Experiences', 'Experiences in Life', 'Dating Advice', 'Dating and Relationships', 'Interpersonal Interaction', 'Survey Question']}
<class 'dict'>
*************************************************************************
processing question number  : 1133
*************************************************************************
http://www.quora.com/My-ex-girlfriend-ended-it-with-me-because-she-believes-our-flame-died-out-I-begged-and-pleaded-with-her-and-she-ended-up-getting-cold-and-blunt-Will-time-and-space-make-her-come-back-to-me

[IndexError('list index out of range',), '', 0]

<class 'dict'>

<class 'dict'>
Respond collect